# Install and Import

In [ ]:
!pip install git+https://github.com/openai/whisper.git
!pip install Resemblyzer
!sudo apt update && sudo apt install ffmpeg
!pip install librosa
!pip install youtube_dl
!pip install datasets
!pip install pydub
!pip3 install spectralcluster

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-tvp2tpet
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-tvp2tpet
  Resolved https://github.com/openai/whisper.git to commit 248b6cb124225dd263bb9bd32d060b6517e067f8
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 22.0 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20230314-py3-none-any.whl size=798075 sha256=47a7015d9737bf56441e533016d19d78a0a437bd88b9a5f89dc7fdca03a69718
  Stored in directory: /tmp/pip-ephem-wheel-cache-75bd2e4i/wheels/8b/6c/d0/622666868c179f156cf595c8b6f06f88bc5d80c4b31dccaa03
Successfully built openai-whisper
Looking in indexes: https://pypi.org/simple, https://us-p

Hit:1 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Get:2 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Hit:3 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Get:4 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Get:5 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease [18.1 kB]
Hit:6 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:8 http://archive.ubuntu.com/ubuntu focal InRelease
Get:9 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:10 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Get:11 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Get:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal/main amd64 Packages [29.5 kB]
Get:13 http://archive.ubuntu.com/ubuntu focal-updates/universe amd64 Packages [1,360 kB]
Get

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 60.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 61.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 60.9 MB/s eta 0:00:00


In [ ]:
import requests
import whisper
import time
import librosa
import soundfile as sf
import re
import os
import youtube_dl
import torch
from transformers import pipeline
from datasets import load_dataset
import numpy as np
import copy
import math
from resemblyzer import preprocess_wav, VoiceEncoder
from pathlib import Path
from pydub import AudioSegment
from spectralcluster import SpectralClusterer
import resemblyzer
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.animation import FuncAnimation
from resemblyzer.audio import sampling_rate
from matplotlib import cm
from time import sleep, perf_counter as timer

from sys import stderr
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
_default_colors = plt.rcParams["axes.prop_cycle"].by_key()["color"]
_my_colors = np.array([
    [0, 127, 70],
    [255, 0, 0],
    [255, 217, 38],
    [0, 135, 255],
    [165, 0, 165],
    [255, 167, 255],
    [97, 142, 151],
    [0, 255, 255],
    [255, 96, 38],
    [142, 76, 0],
    [33, 0, 127],
    [0, 0, 0],
    [183, 183, 183],
    [76, 255, 0],
], dtype=np.float) / 255

<ipython-input-11-74abe47bab8a>:17: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  ], dtype=np.float) / 255


In [ ]:
# Connect Google Drive
from google.colab import drive
drive.mount("/content/drive", force_remount=True) # This will prompt for authorization.

Mounted at /content/drive


In [ ]:
# Initialize the device
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
import ipywidgets as widgets

languages = {"af_za": "Afrikaans", "am_et": "Amharic", "ar_eg": "Arabic", "as_in": "Assamese", "az_az": "Azerbaijani", "be_by": "Belarusian", "bg_bg": "Bulgarian", "bn_in": "Bengali", "bs_ba": "Bosnian", "ca_es": "Catalan", "cmn_hans_cn": "Chinese", "cs_cz": "Czech", "cy_gb": "Welsh", "da_dk": "Danish", "de_de": "German", "el_gr": "Greek", "en_us": "English", "es_419": "Spanish", "et_ee": "Estonian", "fa_ir": "Persian", "fi_fi": "Finnish", "fil_ph": "Tagalog", "fr_fr": "French", "gl_es": "Galician", "gu_in": "Gujarati", "ha_ng": "Hausa", "he_il": "Hebrew", "hi_in": "Hindi", "hr_hr": "Croatian", "hu_hu": "Hungarian", "hy_am": "Armenian", "id_id": "Indonesian", "is_is": "Icelandic", "it_it": "Italian", "ja_jp": "Japanese", "jv_id": "Javanese", "ka_ge": "Georgian", "kk_kz": "Kazakh", "km_kh": "Khmer", "kn_in": "Kannada", "ko_kr": "Korean", "lb_lu": "Luxembourgish", "ln_cd": "Lingala", "lo_la": "Lao", "lt_lt": "Lithuanian", "lv_lv": "Latvian", "mi_nz": "Maori", "mk_mk": "Macedonian", "ml_in": "Malayalam", "mn_mn": "Mongolian", "mr_in": "Marathi", "ms_my": "Malay", "mt_mt": "Maltese", "my_mm": "Myanmar", "nb_no": "Norwegian", "ne_np": "Nepali", "nl_nl": "Dutch", "oc_fr": "Occitan", "pa_in": "Punjabi", "pl_pl": "Polish", "ps_af": "Pashto", "pt_br": "Portuguese", "ro_ro": "Romanian", "ru_ru": "Russian", "sd_in": "Sindhi", "sk_sk": "Slovak", "sl_si": "Slovenian", "sn_zw": "Shona", "so_so": "Somali", "sr_rs": "Serbian", "sv_se": "Swedish", "sw_ke": "Swahili", "ta_in": "Tamil", "te_in": "Telugu", "tg_tj": "Tajik", "th_th": "Thai", "tr_tr": "Turkish", "uk_ua": "Ukrainian", "ur_pk": "Urdu", "uz_uz": "Uzbek", "vi_vn": "Vietnamese", "yo_ng": "Yoruba"}
selection = widgets.Dropdown(
    options=[("Select language", None), ("----------", None)] + sorted([(f"{v} ({k})", k) for k, v in languages.items()]),
    value="ko_kr",
    description='Language:',
    disabled=False,
)

selection

Dropdown(description='Language:', index=39, options=(('Select language', None), ('----------', None), ('Afrika…

In [ ]:
lang = selection.value
language = languages[lang]

assert lang is not None, "Please select a language"
print(f"Selected language: {language} ({lang})")

Selected language: Malay (ms_my)


In [ ]:
# model = whisper.load_model("tiny.en")
# model = whisper.load_model("base.en")
# model = whisper.load_model("small", device=device)
model = whisper.load_model("medium.en")
# model = whisper.load_model("large-v2")

100%|█████████████████████████████████████| 1.42G/1.42G [00:28<00:00, 52.8MiB/s]


In [ ]:
print(
    f"Model is {'multilingual' if model.is_multilingual else 'English-only'} "
    f"and has {sum(np.prod(p.shape) for p in model.parameters()):,} parameters."
)

Model is multilingual and has 240,582,912 parameters.


In [ ]:
options = dict(language=language, beam_size=5, best_of=5)
transcribe_options = dict(task="transcribe", **options)
translate_options = dict(task="translate", **options)

# YouTube Transcription

In [ ]:
"""
3. Click the run button and input your YouTube URL in the box below then click enter.

You can use this one to test: https://www.youtube.com/watch?v=CnT-Na1IeVI

The video will be loaded and the audio extracted (this is usually the longest part of the process.)

Your transcript will appear in the folder on the left (you may have to refresh the folder to see it.)

You can download the file when it's completed and upload it on your video's detail page using "autosync."
"""

# This will prompt you for a YouTube video URL
url = input("Enter a YouTube video URL: ")

# Create a youtube-dl options dictionary
ydl_opts = {
    # Specify the format as bestaudio/best
    'format': 'bestaudio/best',
    # Specify the post-processor as ffmpeg to extract audio and convert to mp3
    'postprocessors': [{
        'key': 'FFmpegExtractAudio',
        'preferredcodec': 'mp3',
        'preferredquality': '192',
    }],
    # Specify the output filename as the video title
    'outtmpl': '%(title)s.%(ext)s',
}

# Download the video and extract the audio
with youtube_dl.YoutubeDL(ydl_opts) as ydl:
    ydl.download([url])

# Get the path of the file
file_path = ydl.prepare_filename(ydl.extract_info(url, download=False))
file_path = file_path.replace('.webm', '.mp3')
file_path = file_path.replace('.m4a', '.mp3')




Enter a YouTube video URL: https://www.youtube.com/watch?v=oZQ-2pI1ehg
[youtube] oZQ-2pI1ehg: Downloading webpage


ERROR: Unable to extract uploader id; please report this issue on https://yt-dl.org/bug . Make sure you are using the latest version; see  https://yt-dl.org/update  on how to update. Be sure to call youtube-dl with the --verbose flag and include its complete output.


DownloadError: ignored

In [ ]:
# Get the duration
duration = librosa.get_duration(filename=file_path)
start = time.time()
result = model.transcribe(file_path)

end = time.time()
seconds = end - start

print("Video length:", duration, "seconds")
print("Transcription time:", seconds)

# Split result["text"]  on !,? and . , but save the punctuation
sentences = re.split("([!?.])", result["text"])

# Join the punctuation back to the sentences
sentences = ["".join(i) for i in zip(sentences[0::2], sentences[1::2])]
text = "\n\n".join(sentences)
for s in sentences:
  print(s)

# Save the file as .txt
name = "".join(file_path) + ".txt"
with open(name, "w") as f:
  f.write(text)

print("\n\n", "-"*100, "\n\nYour transcript is here:", name)

In [ ]:
audio = whisper.load_audio(file_path)

# make log-Mel spectrogram and move to the same device as the model
mel = whisper.log_mel_spectrogram(audio).to(model.device)

# detect the spoken language
_, probs = model.detect_language(mel)

print(f"Detected language: {max(probs, key=probs.get)}")

# transcribing audio files

In [ ]:
# This will create the WhisperAudio files if they don't exist.
folders =  ["WhisperAudio/", "WhisperAudio/ProcessedAudio/", "WhisperAudio/TextFiles/"]
for folder in folders:
  path = "/content/drive/MyDrive/" + folder
  if not os.path.exists(path): # Create the folder if it does not exist
    os.mkdir(path)

In [ ]:
# Assuming the audio files are in a folder called "WhisperAudio" in the root of the drive
audio_folder = "/content/drive/MyDrive/WhisperAudio/"

# Get a list of all the file paths and names in the folder
import os
audio_files = []
audio_names = []
for file in os.listdir(audio_folder):
  if file.endswith(".m4a") or file.endswith(".mp3"):
    audio_files.append(audio_folder + file)
    audio_names.append(file)

for f in audio_files:
  print(f)

if len(audio_files) == 0:
  print("You have no files.")

# Loop through the audio files, split each audio file based on pauses in speech then transcribe them with Whisper.
for i, file in enumerate(audio_files): # For each audio file
  print(f"Processing {audio_names[i]}...")
  # Load the audio file and convert it to 16 kHz mono
  audio, sr = librosa.load(file, sr=16000, mono=True)
  # Detect pauses and split the audio. We use a threshold of -30 dB and a minimum pause length of 0.5 seconds.
  pauses = librosa.effects.split(audio, top_db=30, frame_length=2048, hop_length=128)
  # Transcribe each segment and concatenate the results
  transcription = ""
  for start, end in pauses: # For each segment
    segment = audio[start:end]
    # Save the segment as a temporary wav file
    temp_file = "temp.wav"
    sf.write(temp_file, segment, sr, subtype='PCM_16')
    if os.path.getsize(temp_file) > 10000:
      #continue
      # Transcribe the segment with Whisper
      result = model.transcribe(temp_file)
      text = result["text"]
      # Append the text to the transcription
      print(len(transcription.split(" ")), "words processed")
      transcription += text.strip() + " "
      # Delete the temporary file
      os.remove(temp_file)
  # Print the transcription
  print(f"Transcription of {audio_names[i]}:\n")
  print(transcription)
  print("\n")

  # Convert the spaces between sections into paragraph breaks and save the transcription as a txt document in the same folder as MyAudio.
  transcription = re.sub(r"\s\s+", "\n\n", transcription) # Replace multiple spaces with newlines
  text_file = audio_folder + "/TextFiles/" + audio_names[i][:-4] + ".txt" # Create the text file name
  with open(text_file, "w") as f: # Write the transcription to the text file
    f.write(transcription)
  print(f"Saved transcription as {text_file}")

# Move the audio files to "/content/drive/MyDrive/WhisperAudio/Processed"
import shutil
processed_folder = "/content/drive/MyDrive/WhisperAudio/ProcessedAudio/"
if not os.path.exists(processed_folder): # Create the folder if it does not exist
  os.mkdir(processed_folder)
for file in audio_files: # Move each audio file to the processed folder
  shutil.move(file, processed_folder + os.path.basename(file))
  print(f"Moved {file} to {processed_folder}")

# Video Transcription





In [ ]:
# This will create the WhisperVideo files if they don't exist.
folders =  ["WhisperVideo/", "WhisperVideo/ProcessedVideo/", "WhisperVideo/TextFiles/", "WhisperVideo/AudioFiles/"]
for folder in folders:
  path = "/content/drive/MyDrive/" + folder
  if not os.path.exists(path): # Create the folder if it does not exist
    os.mkdir(path)


In [ ]:
# Get the list of video files from the WhisperVideo folder
target_path = "/content/drive/MyDrive/WhisperVideo/"
video_files = os.listdir(target_path)

# Loop through the video files and transcribe them
for video_file in video_files:

  # Skip the file if it is not a video format
  if not video_file.endswith((".mp4", ".mov", ".avi", ".mkv")):
    continue

  # Extract the audio from the video file using librosa
  video_path = target_path + video_file
  audio_path = "/content/drive/MyDrive/WhisperVideo/AudioFiles/" + video_file[:-4] + ".wav" # Replace the video extension with .wav


  y, sr = librosa.load(video_path, sr=16000) # Load the audio with 16 kHz sampling rate

  sf.write(audio_path, y, sr) # Save the audio as a wav file

  # Transcribe the audio file using Whisper
  result = model.transcribe(audio_path)
  text = result["text"].strip()
  text = text.replace(". ", ".\n\n")

  # Save the transcription as a text file in Google Docs
  text_file = video_file[:-4] + ".txt" # Replace the video extension with .txt
  text_path = "/content/drive/MyDrive/WhisperVideo/TextFiles/" + text_file
  with open(text_path, "w") as f:
    f.write(text)

  # Move the video file to the ProcessedVideo folder
  processed_path = "/content/drive/MyDrive/WhisperVideo/ProcessedVideo/" + video_file
  os.rename(video_path, processed_path)

  # Print a message to indicate the progress
  print(f"Processed {video_file} and saved the transcription as {text_file}")

/usr/local/lib/python3.8/dist-packages/librosa/core/audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")


# Long form Video

In [ ]:
# This will create the WhisperVideo files if they don't exist.
folders =  ["WhisperVideo/", "WhisperVideo/ProcessedVideo/", "WhisperVideo/TextFiles/", "WhisperVideo/AudioFiles/"]
for folder in folders:
  path = "/content/drive/MyDrive/" + folder
  if not os.path.exists(path): # Create the folder if it does not exist
    os.mkdir(path)

In [ ]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"

pipe = pipeline(
  "automatic-speech-recognition",
  model="openai/whisper-medium",
  chunk_length_s=30,
  device=device,
)

def seconds_to_time(seconds):
    hours = int(seconds / 3600)
    minutes = int((seconds % 3600) / 60)
    remaining_seconds = int(seconds % 60)
    hundredths = int((seconds - int(seconds)) * 100)
    return '{:02d}:{:02d}:{:02d}.{:02d}'.format(hours, minutes, remaining_seconds, hundredths)

In [ ]:
# Get the list of video files from the WhisperVideo folder
target_path = "/content/drive/MyDrive/WhisperVideo/"
video_files = os.listdir(target_path)

# Loop through the video files and transcribe them
for video_file in video_files:

    # Skip the file if it is not a video format
    if not video_file.endswith((".mp4", ".mov", ".avi", ".mkv")):
        continue

    print(f"Currently processing {video_file}")

    # Extract the audio from the video file using librosa
    video_path = target_path + video_file
    audio_path = "/content/drive/MyDrive/WhisperVideo/AudioFiles/" + video_file[:-4] + ".wav" # Replace the video extension with .wav
    if 'y' not in locals() and 'y' not in globals():
        y, sr = librosa.load(video_path, sr=16000)

    if not os.path.isfile(audio_path):
        sf.write(audio_path, y, sr) # Save the audio as a wav file

    print(f"{video_file} audio file has been extracted {audio_path}")

    audio_sample = {
        'path': audio_path,
        'array': y,
        'sampling_rate': sr
    }

    trial = 0
    while True:
        try:
            trial = trial + 1
            # we can also return timestamps for the predictions
            prediction_chunk = pipe(audio_sample, return_timestamps=True)["chunks"]
            break
        except Exception as e:
            print(e)
            if trial >= 2:
                break
    if trial >= 2:
        print(f"{audio_path} cannot be processed")
        continue

    timestamp_text = []
    pure_text = ''
    for pc in prediction_chunk:
        time_start = seconds_to_time(pc['timestamp'][0])
        time_end = seconds_to_time(pc['timestamp'][1])
        data = {
            'text': pc['text'],
            'start_time': time_start,
            'end_time': time_end
        }
        timestamp_text.append(data)
        pure_text = pure_text + '\n' + pc['text']

    saved_timestamp_text = ''
    for ts_text in timestamp_text:
        saved_timestamp_text = saved_timestamp_text + f'\n[{ts_text["start_time"]}] [{ts_text["end_time"]}] {ts_text["text"]}\n\n'

    # Save the transcription as a text file in Google Docs
    text_file = video_file[:-4] + ".txt" # Replace the video extension with .txt
    timestamp_text_file = video_file[:-4] + "_timestamp_" + ".txt"
    text_path = "/content/drive/MyDrive/WhisperVideo/TextFiles/" + text_file
    timestamp_text_path = "/content/drive/MyDrive/WhisperVideo/TextFiles/" + timestamp_text_file
    with open(text_path, "w") as f:
        f.write(pure_text)
    with open(timestamp_text_path, "w") as f:
        f.write(saved_timestamp_text)

    # Move the video file to the ProcessedVideo folder
    processed_path = "/content/drive/MyDrive/WhisperVideo/ProcessedVideo/" + video_file
    os.rename(video_path, processed_path)

    # Print a message to indicate the progress
    print(f"Processed {video_file} and saved the transcription as {text_file}")

Currently processing livecall_09feb2020 (240p) Joule Thief.mp4


/usr/local/lib/python3.8/dist-packages/librosa/core/audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")


livecall_09feb2020 (240p) Joule Thief.mp4 audio file has been extracted /content/drive/MyDrive/WhisperVideo/AudioFiles/livecall_09feb2020 (240p) Joule Thief.wav


/usr/local/lib/python3.8/dist-packages/transformers/generation/utils.py:1273: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 448 (`generation_config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Processed livecall_09feb2020 (240p) Joule Thief.mp4 and saved the transcription as livecall_09feb2020 (240p) Joule Thief.txt


# Audio

In [ ]:
# This will create the WhisperVideo files if they don't exist.
folders =  ["WhisperVideo/", "WhisperVideo/ProcessedVideo/", "WhisperVideo/TextFiles/", "WhisperVideo/AudioFiles/"]
for folder in folders:
  path = "/content/drive/MyDrive/" + folder
  if not os.path.exists(path): # Create the folder if it does not exist
    os.mkdir(path)

In [ ]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"

pipe = pipeline(
  "automatic-speech-recognition",
  model="openai/whisper-medium",
  chunk_length_s=30,
  device=device,
)

def seconds_to_time(seconds):
    hours = int(seconds / 3600)
    minutes = int((seconds % 3600) / 60)
    remaining_seconds = int(seconds % 60)
    hundredths = int((seconds - int(seconds)) * 100)
    return '{:02d}:{:02d}:{:02d}.{:02d}'.format(hours, minutes, remaining_seconds, hundredths)

In [ ]:
# Assuming the audio files are in a folder called "WhisperAudio" in the root of the drive
audio_folder = "/content/drive/MyDrive/WhisperVideo/"

# Get a list of all the file paths and names in the folder
import os
audio_files = []
audio_names = []
for file in os.listdir(audio_folder):
  if file.endswith(".m4a") or file.endswith(".mp3"):
    audio_files.append(audio_folder + file)
    audio_names.append(file)

for f in audio_files:
  print(f)

for f in audio_names:
  print(f)

if len(audio_files) == 0:
  print("You have no files.")

/content/drive/MyDrive/WhisperVideo/Jane Zhang 860_1675741911.m4a
Jane Zhang 860_1675741911.m4a


In [ ]:
for i, file in enumerate(audio_files): # For each audio file
    print(f"Processing {audio_names[i]}...")
    # Load the audio file and convert it to 16 kHz mono
    audio, sr = librosa.load(file, sr=16000, mono=True)

    audio_sample = {
            'path': audio_files,
            'array': audio,
            'sampling_rate': sr
        }

    trial = 0
    while True:
        try:
            trial = trial + 1
            # we can also return timestamps for the predictions
            prediction_chunk = pipe(audio_sample, return_timestamps=True)["chunks"]
            break
        except Exception as e:
            print(e)
            if trial >= 2:
                break
    if trial >= 2:
        print(f"{file} cannot be processed")
        continue

    timestamp_text = []
    pure_text = ''
    for pc in prediction_chunk:
        time_start = seconds_to_time(pc['timestamp'][0])
        time_end = seconds_to_time(pc['timestamp'][1])
        data = {
            'text': pc['text'],
            'start_time': time_start,
            'end_time': time_end
        }
        timestamp_text.append(data)
        pure_text = pure_text + '\n' + pc['text']

    saved_timestamp_text = ''
    for ts_text in timestamp_text:
        saved_timestamp_text = saved_timestamp_text + f'\n[{ts_text["start_time"]}] [{ts_text["end_time"]}] {ts_text["text"]}\n\n'

    # Save the transcription as a text file in Google Docs
    text_file = audio_names[i][:-4] + ".txt" # Replace the video extension with .txt
    timestamp_text_file = audio_names[i][:-4]  + "_timestamp_" + ".txt"
    text_path = "/content/drive/MyDrive/WhisperVideo/TextFiles/" + text_file
    timestamp_text_path = "/content/drive/MyDrive/WhisperVideo/TextFiles/" + timestamp_text_file
    with open(text_path, "w") as f:
        f.write(pure_text)
    with open(timestamp_text_path, "w") as f:
        f.write(saved_timestamp_text)

    # Move the video file to the ProcessedVideo folder
    processed_path = "/content/drive/MyDrive/WhisperVideo/AudioFiles/" + audio_names[i]
    os.rename(file, processed_path)

    # Print a message to indicate the progress
    print(f"Processed {file} and saved the transcription as {text_file}")

Processing Jane Zhang 860_1675741911.m4a...


/usr/local/lib/python3.8/dist-packages/librosa/core/audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")
/usr/local/lib/python3.8/dist-packages/transformers/generation/utils.py:1273: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 448 (`generation_config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Processed /content/drive/MyDrive/WhisperVideo/Jane Zhang 860_1675741911.m4a and saved the transcription as Jane Zhang 860_1675741911.txt




# Speaker Diatriazation

In [ ]:
# This will create the WhisperVideo files if they don't exist.
folders =  ["WhisperVideo/", "WhisperVideo/ProcessedVideo/", "WhisperVideo/TextFiles/", "WhisperVideo/AudioFiles/"]
for folder in folders:
  path = "/content/drive/MyDrive/" + folder
  if not os.path.exists(path): # Create the folder if it does not exist
    os.mkdir(path)

In [ ]:
import ipywidgets as widgets

languages = {"wav": "wav", "mp3": "mp3", "m4a": "m4a"}
selection = widgets.Dropdown(
    options=[("Select format", None), ("----------", None)] + sorted([(f"{v} ({k})", k) for k, v in languages.items()]),
    value="wav",
    description='source format:',
    disabled=False,
)

selection

Dropdown(description='source format:', index=4, options=(('Select format', None), ('----------', None), ('m4a …

In [ ]:
# Assuming the audio files are in a folder called "WhisperAudio" in the root of the drive
audio_folder = "/content/drive/MyDrive/WhisperVideo/"

# Get a list of all the file paths and names in the folder
import os
audio_absolute_paths = []
audio_basenames = []
for file in os.listdir(audio_folder):
  if file.endswith(".m4a") or file.endswith(".mp3") or file.endswith(".wav"):
    audio_absolute_paths.append(audio_folder + file)
    audio_basenames.append(file)

for f in audio_absolute_paths:
  print(f)

for f in audio_basenames:
  print(f)

if len(audio_absolute_paths) == 0:
  print("You have no files.")

/content/drive/MyDrive/WhisperVideo/2023_02_06Zhangyan2.wav
/content/drive/MyDrive/WhisperVideo/2023_02_06ZhangYan 1.wav
2023_02_06Zhangyan2.wav
2023_02_06ZhangYan 1.wav


In [ ]:
for idx, audio_path in enumerate(audio_absolute_paths):
    print(f"Now processing {idx}|| {audio_path}")

    track = AudioSegment.from_file(audio_path, format=selection.value)

    wav_filename = audio_folder + os.path.basename(audio_path).replace(selection.value, "wav")

    file_handle = track.export(wav_filename, format='wav')
    # Move the video file to the ProcessedVideo folder
    processed_path = "/content/drive/MyDrive/WhisperVideo/AudioFiles/" + os.path.basename(audio_path)
    os.rename(audio_path, processed_path)

    # Print a message to indicate the progress
    print(f"Processed {audio_path} and saved the transcription as {wav_filename}")

In [ ]:
from typing import Optional, Union
from scipy.ndimage.morphology import binary_dilation
from resemblyzer.hparams import *
from pathlib import Path
from typing import Optional, Union
import numpy as np
import webrtcvad
import librosa
import struct

int16_max = (2 ** 15) - 1

def wav_to_mel_spectrogram(wav):
    """
    Derives a mel spectrogram ready to be used by the encoder from a preprocessed audio waveform.
    Note: this not a log-mel spectrogram.
    """
    frames = librosa.feature.melspectrogram(
        y=wav,
        sr=sampling_rate,
        n_fft=int(sampling_rate * mel_window_length / 1000),
        hop_length=int(sampling_rate * mel_window_step / 1000),
        n_mels=mel_n_channels
    )
    return frames.astype(np.float32).T


def trim_long_silences(wav):
    """
    Ensures that segments without voice in the waveform remain no longer than a
    threshold determined by the VAD parameters in params.py.
    :param wav: the raw waveform as a numpy array of floats
    :return: the same waveform with silences trimmed away (length <= original wav length)
    """
    # Compute the voice detection window size
    samples_per_window = (vad_window_length * sampling_rate) // 1000

    # Trim the end of the audio to have a multiple of the window size
    wav = wav[:len(wav) - (len(wav) % samples_per_window)]

    # Convert the float waveform to 16-bit mono PCM
    pcm_wave = struct.pack("%dh" % len(wav), *(np.round(wav * int16_max)).astype(np.int16))

    # Perform voice activation detection
    voice_flags = []
    vad = webrtcvad.Vad(mode=3)
    for window_start in range(0, len(wav), samples_per_window):
        window_end = window_start + samples_per_window
        voice_flags.append(vad.is_speech(pcm_wave[window_start * 2:window_end * 2],
                                         sample_rate=sampling_rate))
    voice_flags = np.array(voice_flags)

    # Smooth the voice detection with a moving average
    def moving_average(array, width):
        array_padded = np.concatenate((np.zeros((width - 1) // 2), array, np.zeros(width // 2)))
        ret = np.cumsum(array_padded, dtype=float)
        ret[width:] = ret[width:] - ret[:-width]
        return ret[width - 1:] / width

    audio_mask = moving_average(voice_flags, vad_moving_average_width)
    audio_mask = np.round(audio_mask).astype(np.bool)

    # Dilate the voiced regions
    audio_mask = binary_dilation(audio_mask, np.ones(vad_max_silence_length + 1))
    audio_mask = np.repeat(audio_mask, samples_per_window)

    return wav[audio_mask == True]


def normalize_volume(wav, target_dBFS, increase_only=False, decrease_only=False):
    if increase_only and decrease_only:
        raise ValueError("Both increase only and decrease only are set")
    rms = np.sqrt(np.mean((wav * int16_max) ** 2))
    wave_dBFS = 20 * np.log10(rms / int16_max)
    dBFS_change = target_dBFS - wave_dBFS
    if dBFS_change < 0 and increase_only or dBFS_change > 0 and decrease_only:
        return wav
    return wav * (10 ** (dBFS_change / 20))

def self_preprocess_wav(fpath_or_wav: Union[str, Path, np.ndarray], source_sr: Optional[int]=None):
    """
    Applies preprocessing operations to a waveform either on disk or in memory such that
    The waveform will be resampled to match the data hyperparameters.
    :param fpath_or_wav: either a filepath to an audio file (many extensions are supported, not
    just .wav), either the waveform as a numpy array of floats.
    :param source_sr: if passing an audio waveform, the sampling rate of the waveform before
    preprocessing. After preprocessing, the waveform'speaker sampling rate will match the data
    hyperparameters. If passing a filepath, the sampling rate will be automatically detected and
    this argument will be ignored.
    """
    # Load the wav from disk if needed
    if isinstance(fpath_or_wav, str) or isinstance(fpath_or_wav, Path):
        wav, source_sr = librosa.load(str(fpath_or_wav), sr=None)
    else:
        wav = fpath_or_wav

    # Resample the wav
    if source_sr is not None:
        wav = librosa.resample(wav, orig_sr=source_sr, target_sr=sampling_rate)

    # Apply the preprocessing: normalize volume and shorten long silences
    wav = normalize_volume(wav, audio_norm_target_dBFS, increase_only=True)

    return wav

In [ ]:
audio_file_path = r'/content/drive/MyDrive/WhisperVideo/2023_02_06ZhangYan 1.wav'
wav_fpath = Path(audio_file_path)

wav = self_preprocess_wav(wav_fpath, source_sr=16000)


In [ ]:
encoder = VoiceEncoder("cpu")
_, cont_embeds, wav_splits = encoder.embed_utterance(wav, return_partials=True, rate=16)
print(cont_embeds.shape)

Loaded the voice encoder model on cpu in 0.01 seconds.
(7154, 256)


In [ ]:
from spectralcluster import RefinementOptions
from spectralcluster import ThresholdType
from spectralcluster import ICASSP2018_REFINEMENT_SEQUENCE

refinement_options = RefinementOptions(
    gaussian_blur_sigma=1,
    p_percentile=0.5,
    thresholding_soft_multiplier=0.01,
    thresholding_type=ThresholdType.RowMax,
    refinement_sequence=ICASSP2018_REFINEMENT_SEQUENCE)

clusterer = SpectralClusterer(
    min_clusters=2,
    max_clusters=3,
    refinement_options=refinement_options)

labels = clusterer.predict(cont_embeds)

In [ ]:
def create_labelling(labels,wav_splits):
    from resemblyzer.audio import sampling_rate
    times = [((s.start + s.stop) / 2) / sampling_rate for s in wav_splits]
    labelling = []
    start_time = 0

    for i,time in enumerate(times):
        if i>0 and labels[i]!=labels[i-1]:
            temp = [str(labels[i-1]),start_time,time]
            labelling.append(tuple(temp))
            start_time = time
        if i==len(times)-1:
            temp = [str(labels[i]),start_time,time]
            labelling.append(tuple(temp))

    return labelling

labelling = create_labelling(labels,wav_splits)

In [ ]:
labelling

[('1', 0, 1.46),
 ('0', 1.46, 8.96),
 ('1', 8.96, 9.62),
 ('0', 9.62, 12.44),
 ('1', 12.44, 13.46),
 ('0', 13.46, 29.48),
 ('1', 29.48, 34.28),
 ('0', 34.28, 37.52),
 ('1', 37.52, 42.2),
 ('0', 42.2, 43.94),
 ('1', 43.94, 45.62),
 ('0', 45.62, 47.84),
 ('1', 47.84, 49.46),
 ('0', 49.46, 52.46),
 ('1', 52.46, 53.48),
 ('0', 53.48, 55.04),
 ('1', 55.04, 58.7),
 ('0', 58.7, 63.44),
 ('1', 63.44, 63.62),
 ('0', 63.62, 63.74),
 ('1', 63.74, 64.76),
 ('0', 64.76, 69.38),
 ('1', 69.38, 72.38),
 ('0', 72.38, 73.4),
 ('1', 73.4, 76.22),
 ('0', 76.22, 77.0),
 ('1', 77.0, 79.16),
 ('0', 79.16, 79.64),
 ('1', 79.64, 80.12),
 ('0', 80.12, 80.96),
 ('1', 80.96, 81.26),
 ('0', 81.26, 82.1),
 ('1', 82.1, 83.6),
 ('0', 83.6, 86.0),
 ('1', 86.0, 86.6),
 ('0', 86.6, 87.62),
 ('1', 87.62, 89.18),
 ('0', 89.18, 93.68),
 ('1', 93.68, 95.12),
 ('0', 95.12, 96.02),
 ('1', 96.02, 96.14),
 ('0', 96.14, 97.94),
 ('1', 97.94, 100.34),
 ('0', 100.34, 101.72),
 ('1', 101.72, 103.76),
 ('0', 103.76, 106.46),
 ('1', 

# Grammar Error Detection

In [ ]:
def check_grammar(text):
    """
    Checks the grammar of a given text using the LanguageTool API.
    Returns a tuple containing the number of errors found and a list of tuples
    containing the error message and the most likely correction for each error.
    """
    url = 'https://languagetool.org/api/v2/check'
    data = {
        'text': text,
        'language': 'en-US'
    }
    response = requests.post(url, data=data)
    if response.status_code == 200:
        json_data = response.json()
        errors = []
        for match in json_data['matches']:
            message = match['message']
            short_message = match['shortMessage']
            replacements = match['replacements']
            context = match['context']
            if replacements:
                suggestion = replacements[0]['value']

            else:
                suggestion = None
            if context:
                sentence = context['text']
            else:
                sentence = None
            errors.append((message, short_message, suggestion, sentence))

        num_errors = len(errors)
        error_pct = (num_errors / len(text.split())) * 100
        return (errors, num_errors, error_pct)
    else:
        print('Error: API request failed with status code', response.status_code)

In [ ]:
text = "teddy roosevelt was a grate president who did a lot for the usa. His progressive ideas and reforms helped bring america into the 20th centuary, he faced many up hill battle's durring his presidency such as his opposition to trusts and forming of the rough rider to fight in the spanish-american war. He was a vigorous man who worked hard for him polices and beleifs, he even won the nobel piece prize in 1906. Teddys fight against big buisness helped shape americas governmet today, their will ne'er be anouther president like him."


In [ ]:
errors

[('Possible agreement error', 'am'), ('Possible agreement error', 'is')]

In [ ]:

errors, num_errors, error_pct = check_grammar(text)
if num_errors:
    print(f'{num_errors} grammar errors found ({error_pct:.2f}%):')
    for idx, (message, short_message, correction, sentence) in enumerate(errors):
        print(f"ERROR: {idx} {short_message}")
        print(f"CONTEXT: {sentence}")
        print(f"MESSAGE: {message}")
        print(f"CORRECTION: {correction}")

else:
    print('No grammar errors found.')

15 grammar errors found (16.30%):
ERROR: 0 
CONTEXT: teddy roosevelt was a grate president who did...
MESSAGE: This sentence does not start with an uppercase letter.
CORRECTION: Teddy
ERROR: 1 Spelling mistake
CONTEXT: teddy roosevelt was a grate president who did a lot for...
MESSAGE: Possible spelling mistake found.
CORRECTION: Roosevelt
ERROR: 2 Spelling mistake
CONTEXT: ...a grate president who did a lot for the usa. His progressive ideas and reforms help...
MESSAGE: Possible spelling mistake found.
CORRECTION: USA
ERROR: 3 Spelling mistake
CONTEXT: ...gressive ideas and reforms helped bring america into the 20th centuary, he faced many u...
MESSAGE: Possible spelling mistake found.
CORRECTION: America
ERROR: 4 Spelling mistake
CONTEXT: ...orms helped bring america into the 20th centuary, he faced many up hill battle's durring...
MESSAGE: Possible spelling mistake found.
CORRECTION: century
ERROR: 5 Spelling mistake
CONTEXT: ...entuary, he faced many up hill battle's durring his pr

# Server

In [ ]:
!pip install colabcode
!pip install fastapi